In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
!ls

datalab  drive


In [0]:
import os
os.chdir("drive/app/pytorch_cnn")

In [5]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 484.0MB 24kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5b806000 @  0x7fc4e1e761c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 14.6MB/s 
    100% |████████████████████████████████| 2.0MB 17.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
print(torch.__version__)

0.4.0


In [7]:
#The output of torchvision datasets are PILImage images of range [0, 1]. We transform them to Tensors of normalized range [-1, 1].

transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=125,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=125,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [8]:
#Let us show some of the training images, for fun.
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    print(npimg.shape)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    npimg = np.transpose(npimg, (1, 2, 0))
    print(npimg.shape)

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(1)))

(3, 546, 274)
(546, 274, 3)
 frog


ValueError: ignored

In [9]:
# ((images[0:,:,:]/2)+0.5)*255
images.shape

torch.Size([125, 3, 32, 32])

In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
    self.batch_norm = nn.BatchNorm2d(32)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
    self.batch_norm1 = nn.BatchNorm2d(64)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
    self.batch_norm2 = nn.BatchNorm2d(128)
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
    self.batch_norm3 = nn.BatchNorm2d(256)
    self.fc1 = nn.Linear(256 * 8 * 8, 512)
  #         self.batch_norm4 = nn.BatchNorm1d(512)
    self.dropout3 = nn.Dropout2d(p=0.5)
    self.fc2 = nn.Linear(512, 256)
  #         self.batch_norm5 = nn.BatchNorm1d(256)
    self.dropout4 = nn.Dropout2d(p=0.5)
    self.fc3 = nn.Linear(256, 10)

  def forward(self, x):
    x = self.pool(self.batch_norm1(F.relu(self.conv2(self.batch_norm(F.relu(self.conv1(x))))))) 
    x = self.pool(self.batch_norm3(F.relu(self.conv4(self.batch_norm2(F.relu(self.conv3(x)))))))
#     x = self.dropout1(self.pool(F.relu(self.conv2(self.batch_norm(F.relu(self.conv1(x)))))))
#     x = self.dropout2(self.pool(F.relu(self.conv41(self.batch_norm2(F.relu(self.conv3(x)))))))
    
    x = x.view(-1, 256 * 8 * 8)
    x = self.dropout3(F.relu(self.fc1(x)))
    x = self.dropout4(F.relu(self.fc2(x)))
    x = self.fc3(x)
    return x

net = Net()

In [0]:
 import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001, amsgrad=False)
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assume that we are on a CUDA machine, then this should print a CUDA device:

print(device)
net.to(device)

cuda:0


Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch_norm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16384, out_features=512, bias=True)
  (dropout3): Dropout2d(p=0.5)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (dropout4): Dropout2d(p=0.5)
  (fc3): Linear(in_feat

In [17]:
for epoch in range(50):  # loop over the dataset multiple times
  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    # get the inputs
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
  
    # print statistics
    running_loss += loss.item()
    if i % 400 == 399:    # print every 391 mini-batches
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 400))
      train_accuracy()
      test_accuracy()
      running_loss = 0.0

print('Finished Training')

[1,   400] loss: 1.602
Accuracy of the network on the 50000 train images: 54 %
Accuracy of the network on the 10000 test images: 54 %
[2,   400] loss: 1.128
Accuracy of the network on the 50000 train images: 65 %
Accuracy of the network on the 10000 test images: 64 %
[3,   400] loss: 0.926
Accuracy of the network on the 50000 train images: 71 %
Accuracy of the network on the 10000 test images: 71 %
[4,   400] loss: 0.823
Accuracy of the network on the 50000 train images: 74 %
Accuracy of the network on the 10000 test images: 72 %
[5,   400] loss: 0.756
Accuracy of the network on the 50000 train images: 76 %
Accuracy of the network on the 10000 test images: 74 %
[6,   400] loss: 0.710
Accuracy of the network on the 50000 train images: 77 %
Accuracy of the network on the 10000 test images: 76 %
[7,   400] loss: 0.677
Accuracy of the network on the 50000 train images: 78 %
Accuracy of the network on the 10000 test images: 76 %
[8,   400] loss: 0.645
Accuracy of the network on the 50000 tr

[23,   400] loss: 0.505
Accuracy of the network on the 50000 train images: 84 %
Accuracy of the network on the 10000 test images: 81 %
[24,   400] loss: 0.501
Accuracy of the network on the 50000 train images: 84 %
Accuracy of the network on the 10000 test images: 81 %
[25,   400] loss: 0.499
Accuracy of the network on the 50000 train images: 84 %
Accuracy of the network on the 10000 test images: 80 %
[26,   400] loss: 0.494
Accuracy of the network on the 50000 train images: 83 %
Accuracy of the network on the 10000 test images: 81 %
[27,   400] loss: 0.489
Accuracy of the network on the 50000 train images: 84 %
Accuracy of the network on the 10000 test images: 81 %
[28,   400] loss: 0.488
Accuracy of the network on the 50000 train images: 84 %
Accuracy of the network on the 10000 test images: 81 %
[29,   400] loss: 0.484
Accuracy of the network on the 50000 train images: 83 %
Accuracy of the network on the 10000 test images: 80 %
[30,   400] loss: 0.482
Accuracy of the network on the 

[45,   400] loss: 0.457
Accuracy of the network on the 50000 train images: 85 %
Accuracy of the network on the 10000 test images: 82 %
[46,   400] loss: 0.453
Accuracy of the network on the 50000 train images: 85 %
Accuracy of the network on the 10000 test images: 82 %
[47,   400] loss: 0.459
Accuracy of the network on the 50000 train images: 86 %
Accuracy of the network on the 10000 test images: 82 %
[48,   400] loss: 0.456
Accuracy of the network on the 50000 train images: 85 %
Accuracy of the network on the 10000 test images: 81 %
[49,   400] loss: 0.449
Accuracy of the network on the 50000 train images: 85 %
Accuracy of the network on the 10000 test images: 82 %
[50,   400] loss: 0.450
Accuracy of the network on the 50000 train images: 85 %
Accuracy of the network on the 10000 test images: 82 %
Finished Training


In [13]:
dataiter = iter(testloader)
images, labels = dataiter.next()
images, labels = images.to(device), labels.to(device)
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

GroundTruth:    cat  ship  ship plane


In [14]:
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Predicted:   deer  bird   cat plane


In [0]:
def train_accuracy():
  correct = 0
  total = 0
  with torch.no_grad():
      for data in trainloader:
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the 50000 train images: %d %%' % (
      100 * correct / total))



In [0]:
def test_accuracy():
  correct = 0
  total = 0
  with torch.no_grad():
      for data in testloader:
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = net(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the 10000 test images: %d %%' % (
      100 * correct / total))

In [65]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 81 %
Accuracy of   car : 88 %
Accuracy of  bird : 66 %
Accuracy of   cat : 64 %
Accuracy of  deer : 71 %
Accuracy of   dog : 75 %
Accuracy of  frog : 78 %
Accuracy of horse : 84 %
Accuracy of  ship : 73 %
Accuracy of truck : 91 %


In [0]:
!kill -9 -1 